# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [5]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [7]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [10]:
# Your code here
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train = pd.DataFrame(X_train_scaled, columns=X.columns)
X_val = pd.DataFrame(X_val_scaled, columns=X.columns)

In [11]:
# Your code here

# Create a LinearRegression model and fit it on scaled training data
regression = LinearRegression()
regression.fit(X_train, y_train)

# Calculate a baseline r-squared score on training data
baseline_train = regression.score(X_train, y_train)
baseline_val = regression.score(X_val, y_val)

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [13]:
# Your code here

# Set up data structure
interactions = []

# Find combinations of columns and loop over them
col_comb = list(combinations(X_train.columns, 2))
for (a, b) in col_comb:
    # Make copies of X_train and X_val
    X_train_copy = X_train.copy()
    X_val_copy = X_val.copy()
    
    # Add interaction term to data
    X_train_copy['interaction'] = X_train_copy[a] * X_train_copy[b]
    X_val_copy['interaction'] = X_val_copy[a] * X_val_copy[b]
    # Find r-squared score (fit on training data, evaluate on validation data)
    r_squared = LinearRegression().fit(X_train_copy, y_train).score(X_val_copy, y_val)
    
    # Append to data structure
    interactions.append(((a, b), r_squared))
    
# Sort and subset the data structure to find the top 7
interactions.sort(key=lambda x:x[1],reverse=True)
print(interactions[:7])

[(('LotArea', '1stFlrSF'), 0.7211105666140571), (('LotArea', 'TotalBsmtSF'), 0.707164920705011), (('LotArea', 'GrLivArea'), 0.6690980823779022), (('LotArea', 'Fireplaces'), 0.6529699515652586), (('2ndFlrSF', 'TotRmsAbvGrd'), 0.647299489040519), (('OverallCond', 'TotalBsmtSF'), 0.642901987923377), (('OverallQual', '2ndFlrSF'), 0.6422324294284367)]


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [15]:
# Your code here

# Loop over top 7 interactions
for interaction in interactions[:7]:
    # Extract column names from data structure
    (a, b) = interaction[0]
    # Construct new column name
    name = a + '_' + b
    # Add new column to X_train and X_val
    X_train[name] = X_train[a] * X_train[b]
    X_val[name] = X_val[a] * X_val[b]


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [17]:
# Your code here

# Set up data structure
polynomials = []
# Loop over all columns
for col in X_train.columns:
    # Loop over degrees 2, 3, 4
    for deg in (2, 3, 4):
        # Make a copy of X_train and X_val
        X_train_copy = X_train.copy()
        X_val_copy = X_val.copy()
        
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(deg, include_bias=False)
        
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        col_transformed_train = pd.DataFrame(poly.fit_transform(X_train_copy[[col]]))
        col_transformed_val = pd.DataFrame(poly.transform(X_val_copy[[col]]))
        
        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
        features_train = pd.concat([X_train_copy.drop(col, axis=1), col_transformed_train], axis=1)
        features_val = pd.concat([X_val_copy.drop(col, axis=1), col_transformed_val], axis=1)
        
        # Find r-squared score on validation
        r_squared = LinearRegression().fit(X_train_copy, y_train).score(X_val_copy, y_val)
        # Append to data structure
        polynomials.append((col, deg, r_squared))

# Sort and subset the data structure to find the top 7
polynomials.sort(key=lambda x:x[1],reverse=True)
print(polynomials[:7])

[('LotArea', 4, 0.746820544030099), ('OverallQual', 4, 0.746820544030099), ('OverallCond', 4, 0.746820544030099), ('TotalBsmtSF', 4, 0.746820544030099), ('1stFlrSF', 4, 0.746820544030099), ('2ndFlrSF', 4, 0.746820544030099), ('GrLivArea', 4, 0.746820544030099)]


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [19]:
# Your code here
top_polynomials = pd.DataFrame(polynomials[:7], columns=["Column", "Degree", "R^2"])
top_polynomials
# Filter out duplicates
top_polynomials.drop_duplicates(subset="Column", inplace=True)
top_polynomials
# Loop over remaining results
for (col, degree, _) in top_polynomials.values:
    # Create polynomial terms
    poly = PolynomialFeatures(degree, include_bias=False)
    train_poly = pd.DataFrame(poly.fit_transform(X_train[[col]]), columns=poly.get_feature_names_out([col]))
    val_poly = pd.DataFrame(poly.transform(X_val[[col]]), columns=poly.get_feature_names_out([col]))
    # Concat new polynomials to X_train and X_val
    X_train = pd.concat([X_train.drop(col, axis=1), train_poly], axis=1)
    X_val = pd.concat([X_val.drop(col, axis=1), val_poly], axis=1)    

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [21]:
# Your code here
X_train.head()

,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF,...,1stFlrSF^3,1stFlrSF^4,2ndFlrSF,2ndFlrSF^2,2ndFlrSF^3,2ndFlrSF^4,GrLivArea,GrLivArea^2,GrLivArea^3,GrLivArea^4
0,0.250689,0.327629,-0.994820,0.092318,0.073336,-0.057254,0.114116,0.316257,0.325573,-0.125956,...,-0.521250,0.419496,1.261552,1.591512,2.007775,2.532911,0.499114,0.249115,0.124337,0.062058
1,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,0.043694,0.175804,0.295392,-0.426859,-0.510770,...,0.264125,0.169465,-0.808132,0.653077,-0.527772,0.426509,-0.247249,0.061132,-0.015115,0.003737
2,-0.981739,-1.105931,-0.994820,0.081045,0.003094,0.232730,0.245060,0.793375,-0.016386,0.672141,...,-0.035611,0.011716,-0.808132,0.653077,-0.527772,0.426509,-0.944766,0.892583,-0.843282,0.796704
3,-0.981739,-1.134602,0.588023,0.230591,0.128368,0.433899,-0.222636,0.793375,-0.442323,0.672141,...,-0.225906,0.137585,-0.808132,0.653077,-0.527772,0.426509,-1.146010,1.313340,-1.505101,1.724862
4,0.250689,-2.281450,-0.994820,0.014341,0.027589,0.005250,0.010842,0.138010,-3.302627,-0.860799,...,-2.278716,2.998612,0.550523,0.303075,0.166850,0.091854,-0.481708,0.232043,-0.111777,0.053844


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [24]:
# Your code here
reg = LinearRegression()
reg.fit(X_train, y_train)

print("Train R-Squared:", reg.score(X_train, y_train))
print("Validation R-Squared: ", reg.score(X_val, y_val))

Train R-Squared: 0.8788607883639323
Validation R-Squared:  -20.09861254944085


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [27]:
# Your code here
for n in range(1, 20):
    rfe = RFE(LinearRegression(), n_features_to_select=n*2)
    X_rfe_train = rfe.fit_transform(X_train, y_train)
    X_rfe_val = rfe.transform(X_val)

    lr = LinearRegression()
    lr.fit(X_rfe_train, y_train)

    print("Train R^2:", lr.score(X_rfe_train, y_train))
    print("Test R^2: ", lr.score(X_rfe_val, y_val))
    print(f"Using {n*2} out of {X_train.shape[1]} features")
    print()

Train R^2: 0.7016756546164582
Test R^2:  0.6066047409179585
Using 2 out of 38 features

Train R^2: 0.7613382811926305
Test R^2:  0.6145675279067211
Using 4 out of 38 features

Train R^2: 0.8025649160167796
Test R^2:  0.6451196115158557
Using 6 out of 38 features

Train R^2: 0.8150363311499462
Test R^2:  0.6536945906395959
Using 8 out of 38 features

Train R^2: 0.8191862278324273
Test R^2:  0.674347615986074
Using 10 out of 38 features

Train R^2: 0.8204988167998999
Test R^2:  0.6713872111439736
Using 12 out of 38 features

Train R^2: 0.8495318516143885
Test R^2:  -0.37393758580721403
Using 14 out of 38 features

Train R^2: 0.8561451376337077
Test R^2:  -0.5346481249170734
Using 16 out of 38 features

Train R^2: 0.8563326757174328
Test R^2:  -0.7239730921996586
Using 18 out of 38 features

Train R^2: 0.8739154002825809
Test R^2:  -35.24588722374384
Using 20 out of 38 features

Train R^2: 0.8739159729873535
Test R^2:  -35.26526837430693
Using 22 out of 38 features

Train R^2: 0.876516400

Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [29]:
# Your code here
for alpha in [1, 10, 100, 1000, 10000]:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)

    print("Train R^2:", lasso.score(X_train, y_train))
    print("Validation R^2: ", lasso.score(X_val, y_val))
    print(f"Using { 26 - (sum(abs(lasso.coef_) < 10**(-10)))} out of {X_train.shape[1]} features")
    print("and an alpha of", alpha)
    print()

Train R^2: 0.8788147491291005
Validation R^2:  -19.57146596556102
Using 26 out of 38 features
and an alpha of 1

Train R^2: 0.8787918494675016
Validation R^2:  -19.649255896133774
Using 26 out of 38 features
and an alpha of 10

Train R^2: 0.8781801955905787
Validation R^2:  -19.773609887006337
Using 20 out of 38 features
and an alpha of 100

Train R^2: 0.8703164047633913
Validation R^2:  -16.90140646492175
Using 14 out of 38 features
and an alpha of 1000

Train R^2: 0.7954415133340873
Validation R^2:  -20.650854157484858
Using 3 out of 38 features
and an alpha of 10000



Compare the results. Which features would you choose to use?

In [31]:
# Your written answer here

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [33]:
# Scale X_test
X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled, columns=X.columns)

# Add interactions to X_test
for record in interactions[:7]:
    col1, col2 = record[0]
    new_col_name = col1 + "_" + col2
    X_test[new_col_name] = X_test[col1] * X_test[col2]

# Add polynomials to X_val
for (col, degree, _) in top_polynomials.values:
    poly = PolynomialFeatures(degree, include_bias=False)
    poly_test = pd.DataFrame(poly.fit_transform(X_test[[col]]),
                                        columns=poly.get_feature_names_out([col]))
    X_test = pd.concat([X_test.drop(col, axis=1), poly_test], axis=1)

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [55]:
# Your code here
final_model = Lasso(alpha=10000)
final_model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

print("R-Squared:", final_model.score(X_test, y_test))
print("MSE:", mean_squared_error(y_test, final_model.predict(X_test)))

R-Squared: 0.7753027817785102
MSE: 1574073456.8424664


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.